In [1]:
# Imports & Parameters
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Parameters
START_DATE = '2020-01-01'
TICKERS = [
    'BTC-USD', 'ETH-USD', 
    #'XRP-USD', 'BNB-USD', 'SOL-USD', 
    #'TRX-USD', 'ADA-USD', 'DOGE-USD', 'ZEC-USD', 'LINK-USD'
]

SUB_PLOTS_X = 1
SUB_PLOTS_Y = 2

In [3]:
# Download data for all tickers
data = yf.download(TICKERS, start=START_DATE, auto_adjust=True, progress=False)

# Returns
returns = data['Close'].pct_change()
returns.columns = pd.MultiIndex.from_product([['Returns'], returns.columns])

# Log Returns
log_returns = np.log(data['Close'] / data['Close'].shift(1))
log_returns.columns = pd.MultiIndex.from_product([['LogReturns'], log_returns.columns])

# 20-day momentum (cumulative return over past 20 days)
momentum_20d = data['Close'].pct_change(periods=20)
momentum_20d.columns = pd.MultiIndex.from_product([['Momentum20D'], momentum_20d.columns])

# Concatenate with original data
data = pd.concat([data, returns, log_returns, momentum_20d], axis=1)